In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

#DATA AUGUMENTATION
train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.2
)

#DATA PRE-PROCESSING
train_data = train_gen.flow_from_directory(
    "/home/sk/Desktop/crop_disease_dataset/Maize_dataset/data",
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    subset = 'training',
    class_mode = 'categorical'
)

val_data = train_gen.flow_from_directory(
    "/home/sk/Desktop/crop_disease_dataset/Maize_dataset/data",
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    subset = 'validation',
    class_mode = 'categorical'
)

Found 3352 images belonging to 4 classes.
Found 836 images belonging to 4 classes.


In [3]:
# This should be mapped as per order in actual dataset

class_weights = {
    0: 0.8017,  
    1: 0.9010,  
    2: 1.8240, 
    3: 0.9136   
}


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# To stop overfitting

early_stop = EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=5,              # Stop if no improvement for 5 epochs
    restore_best_weights=True # Restore the weights of the best epoch
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,     # Reduce LR by factor
    patience=3,     # Wait 3 epochs before reducing
    min_lr=1e-6
)



history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    batch_size=BATCH_SIZE
)

# Save the model
model.save("maize_disease_model.h5")


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 235ms/step - accuracy: 0.8452 - loss: 0.5239 - val_accuracy: 0.8696 - val_loss: 0.4098
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 233ms/step - accuracy: 0.8434 - loss: 0.3758 - val_accuracy: 0.8864 - val_loss: 0.3337
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 24s 231ms/step - accuracy: 0.8496 - loss: 0.4593 - val_accuracy: 0.8792 - val_loss: 0.2660
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 233ms/step - accuracy: 0.8398 - loss: 0.4494 - val_accuracy: 0.8696 - val_loss: 0.3745
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 233ms/step - accuracy: 0.8431 - loss: 0.4860 - val_accuracy: 0.6017 - val_loss: 4.9991
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 233ms/step - accuracy: 0.8311 - loss: 0.4881 - val_accuracy: 0.8600 - val_loss: 0.4491
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 234ms/step - accuracy: 0.8368 - loss: 0.4698 - val_accuracy: 0.7356 - val_loss: 1.0143
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 234ms/step - accuracy: 0.8502 - loss: 0

In [7]:
from tensorflow.keras.models import load_model

# Load your trained maize model
model = load_model("maize_disease_model.h5")
class_labels = ['Northern_Leaf_Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']


In [18]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path, threshold=0.75):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    preds = model.predict(img_array, verbose=0)
    pred_class = class_labels[np.argmax(preds)]
    confidence = np.max(preds)

    if confidence >= threshold:
        pred_class = class_labels[np.argmax(preds)]

    else:
        pred_class = "Unknown Disease"

    return pred_class, confidence


In [19]:
predict_image("/home/sk/Desktop/crop_disease_dataset/Onion_Dataset/onion datasets/Caterpillar-P/8145132_1680251728524_640x480.jpg")

('Unknown Disease', np.float32(0.3786256))